# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import geoviews as gv
import geoviews.feature as gf
from geoviews import opts



# Import API key
from api_keys_v import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame

file_path = Path('../WeatherPy/output_data/cities.csv')
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,arraial do cabo,-22.9661,-42.0278,23.53,82,15,1.30,BR,2023-04-06
1,1,bagotville,-28.9833,153.4167,19.80,77,33,2.16,AU,2023-04-06
2,2,kaitangata,-46.2817,169.8464,8.85,78,44,3.63,NZ,2023-04-06
3,3,east london,-33.0153,27.9116,25.52,61,0,5.14,ZA,2023-04-06
4,4,bluff,-46.6000,168.3333,9.86,53,75,5.63,NZ,2023-04-06


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

cities_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    alpha = 0.5,
    hover_cols=["City", "Humidity"]
)

cities_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities = city_data_df[
    (city_data_df['Max Temp'] < 30) & 
    (city_data_df['Max Temp'] > 25) &
    (city_data_df['Wind Speed'] <= 5) &
    (city_data_df['Cloudiness'] >= 20) & 
    (city_data_df['Cloudiness'] <= 50) &  
    (city_data_df['Humidity'] >= 50) & 
    (city_data_df['Humidity'] <= 80)
]

# Drop any rows with null values
filtered_cities = filtered_cities.dropna()

# Reset the index
filtered_cities = filtered_cities.reset_index()

# Display sample data
print(f"Number of cities in the list: {len(filtered_cities)}")
filtered_cities

Number of cities in the list: 11


,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,53,53,bengkulu,-3.8004,102.2655,28.58,68,25,3.35,ID,2023-04-06
1,60,60,cairns,-16.9167,145.7667,27.97,78,40,1.54,AU,2023-04-06
2,63,63,olinda,-8.0089,-34.8553,26.85,74,40,3.09,BR,2023-04-06
3,133,133,dwarka,22.2394,68.9678,28.05,50,46,4.91,IN,2023-04-06
4,149,149,labuhan,-6.8844,112.2051,29.39,70,40,2.52,ID,2023-04-06
5,152,152,san andres,12.5847,-81.7006,27.01,74,40,4.12,CO,2023-04-06
6,235,235,grand-lahou,5.1367,-5.0242,28.96,71,38,2.48,CI,2023-04-06
7,366,366,sabang,5.8933,95.3214,28.24,73,44,1.85,ID,2023-04-06
8,409,409,bonthe,7.5264,-12.5050,27.63,68,40,0.90,SL,2023-04-06
9,479,479,sassandra,4.9500,-6.0833,28.98,70,48,1.78,CI,2023-04-06


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = hotel_df.assign(**{'Hotel Name': ''})

# Reset the index
hotel_df = hotel_df.reset_index()

# Display sample data
print(f"Number of cities in the list: {len(hotel_df)}")
hotel_df

Number of cities in the list: 11


,index,City,Country,Lat,Lng,Humidity,Hotel Name
0,0,bengkulu,ID,-3.8004,102.2655,68,
1,1,cairns,AU,-16.9167,145.7667,78,
2,2,olinda,BR,-8.0089,-34.8553,74,
3,3,dwarka,IN,22.2394,68.9678,50,
4,4,labuhan,ID,-6.8844,112.2051,70,
5,5,san andres,CO,12.5847,-81.7006,74,
6,6,grand-lahou,CI,5.1367,-5.0242,71,
7,7,sabang,ID,5.8933,95.3214,73,
8,8,bonthe,SL,7.5264,-12.5050,68,
9,9,sassandra,CI,4.9500,-6.0833,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "filter": "categories=accommodation.hotel",
    "bias": f"proximity:{'Lat'},{'Lng'}",
    "limit": 1,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    params["categories"] = "accommodation.hotel"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

hotel_df

# Display sample data
hotel_df

Starting hotel search
bengkulu - nearest hotel: Dena Hotel
cairns - nearest hotel: Cairns Queens Court Accommodation
olinda - nearest hotel: Fidji Motel
dwarka - nearest hotel: The Dwarika Hotel
labuhan - nearest hotel: No hotel found
san andres - nearest hotel: Hotel Decameron Isleño
grand-lahou - nearest hotel: Cap Lahou
sabang - nearest hotel: Sabang Hill
bonthe - nearest hotel: No hotel found
sassandra - nearest hotel: Hotel Le Pollet
miri - nearest hotel: Kenyalang Suites Miri


,index,City,Country,Lat,Lng,Humidity,Hotel Name
0,0,bengkulu,ID,-3.8004,102.2655,68,Dena Hotel
1,1,cairns,AU,-16.9167,145.7667,78,Cairns Queens Court Accommodation
2,2,olinda,BR,-8.0089,-34.8553,74,Fidji Motel
3,3,dwarka,IN,22.2394,68.9678,50,The Dwarika Hotel
4,4,labuhan,ID,-6.8844,112.2051,70,No hotel found
5,5,san andres,CO,12.5847,-81.7006,74,Hotel Decameron Isleño
6,6,grand-lahou,CI,5.1367,-5.0242,71,Cap Lahou
7,7,sabang,ID,5.8933,95.3214,73,Sabang Hill
8,8,bonthe,SL,7.5264,-12.5050,68,No hotel found
9,9,sassandra,CI,4.9500,-6.0833,70,Hotel Le Pollet


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
hotels_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    alpha = 0.5,
    hover_cols=["City", "Humidity", "Hotel Name", "Country"]
)

# Display the map
hotels_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)